In [2]:
%load_ext autoreload
%autoreload 2
import glob
import warnings

import LabelLib as ll  # библиотека для моделирования меток облаком
import matplotlib.pyplot as plt
import MDAnalysis as mda  # анализ мд
import nglview as nv  # библиотека для визуализации молекул
import numpy as np  # для математики
import pynucl
from MDAnalysis.analysis import align
from MDAnalysis.coordinates.memory import MemoryReader
from scipy.spatial.distance import cdist
from tqdm.auto import tqdm

/opt/miniconda3/envs/moldyn/lib/python3.7/site-packages/Bio/Align/substitution_matrices/__init__.py:21: BiopythonExperimentalWarning: Bio.Align.substitution_matrices is an experimental module which may still undergo significant changes. In particular, the location of this module may change, and the Array class defined in this module may be moved to other existing or new modules in Biopython.
  BiopythonExperimentalWarning)


In [7]:
def create_cloud_AV1(
    atoms,
    Dye_attachment_coordinat,
    linker_length_Dye=20,
    linker_width=1,
    dye_radius=7,
    simulation_grid_spacing=1,
    Emean_list=[],
    meanDist_list=[],
):
    av1 = ll.dyeDensityAV1(
        atoms,
        Dye_attachment_coordinat,
        linker_length_Dye,
        linker_width,
        dye_radius,
        simulation_grid_spacing,
    )
    if av1.points().size == 0:
        Emean = None
        meanDist = None
        Emean_list.append(Emean)
        meanDist_list.append(meanDist)
    else:
        pass
    return av1


def labellib_cloud_av1(
    u,
    chain_Dye="L",
    attach_n_Dye=15,
    linker_length_Dye=20,
    dye_radius=7,
    linker_width=1,
    simulation_grid_spacing=1,
    Emean_list=[],
    meanDist_list=[],
):
    atoms, Dye_attachment_coordinat = setings_label(
        u, chain_Dye=chain_Dye, attach_n_Dye=attach_n_Dye
    )

    av1 = create_cloud_AV1(
        atoms=atoms,
        Dye_attachment_coordinat=Dye_attachment_coordinat,
        linker_length_Dye=linker_length_Dye,
        linker_width=linker_width,
        dye_radius=dye_radius,
        simulation_grid_spacing=simulation_grid_spacing,
        Emean_list=Emean_list,
        meanDist_list=meanDist_list,
    )
    #     if av1.points().size == 0:
    #         av1 = 0

    return av1, Dye_attachment_coordinat


def labellib_length_calculating_av1(
    universe,
    chain_Cy3="L",
    attach_n_Cy3=24,
    linker_length_Cy3=16,
    linker_width=1,
    dye_radius=7,
    simulation_grid_spacing=1,
    pqr1_name=None,
    cvFrac=0.3,
    cvRadius=10.6,
    random_g=1000,
):
    """
    universe -- передаем команду с pdb файлом + фаил динамики
    chain_Cy3 -- названия цепочки 1
    attach_n_Cy3 -- номер нуклеотида на который мы прикрепляем метку на цепочке chain_Cy3
    linker_length_Cy3 -- длина линкеров
    linker_width -- ширина линкера
    dye_radius_Cy3 -- радиусы наших меток
    simulation_grid_spacing -- grid step
    surface_weight -- параметр для перевзвешивания позиций меток (пока не работает)
    random_g -- кол-во случайно взятых позиций из облака labelLib
    """
    u = universe

    av1, Cy3_attachment_point = labellib_cloud_av1(
        u,
        chain_Dye=chain_Cy3,
        attach_n_Dye=attach_n_Cy3,
        linker_length_Dye=linker_length_Cy3,
        dye_radius=dye_radius,
        linker_width=linker_width,
        simulation_grid_spacing=simulation_grid_spacing,
    )

    Dye_distances, weights = new_weigth(
        av1, Dye_attachment_coordinat=Cy3_attachment_point, s=4, r0=14
    )
    if not pqr1_name is None:
        savePqr_2(
            f"{pqr1_name}" + ".pqr",
            grid=av1,
            weights=weights,
        )
    return Dye_distances, av1, weights

In [39]:
resid_list = [10, 15, 20, 25, 30, 35]
rad_list = [24, 16, 16, 24, 24, 24]
cloud_dict = {}
dna_nu = mda.Universe("PDB/nucl_DNA_H1_linkered.pdb")
for pos, rad in zip(resid_list, rad_list):
    Dye_distances, av1, weights = labellib_length_calculating_av1(
        dna_nu,
        chain_Cy3="J",
        attach_n_Cy3=pos,
        linker_length_Cy3=rad,
        linker_width=1,
        dye_radius=7,
        pqr1_name=f"pqr_file/AV_Cy3_pos_{pos}",
    )
    cloud_dict[pos] = av1

In [49]:
na_nu = mda.Universe("PDB/nucl_DNA_H1_linkered.pdb")
view = nv.show_mdanalysis(dna_nu)
view 

NGLWidget()

In [50]:
color = ['#CCFF33', '#9EF01A','#38B000','#008000','#006400', '#004B23']
for i, cloud in enumerate(sorted(glob.glob('pqr_file/AV*.pqr'))):
    view.add_component(nv.FileStructure(cloud))
    view[-1].clear()
    view[-1].add_surface(color=color[i],opacity=0.5)

In [9]:
H1_cent = dna_nu.select_atoms('segid u').center_of_geometry()

In [10]:
data_analysis ={}
for key in cloud_dict.keys():
    dist_l = []
    cloud = cloud_dict[key].points().T[:,:-1]
    for cloud_point in cloud: 
        dist_l.append(np.linalg.norm(H1_cent - cloud_point)) 
        data_analysis[key] = (max(dist_l)- min(dist_l))/2


In [11]:
data_analysis

{10: 15.335511852830964,
 15: 11.962345821723774,
 20: 11.664988977647134,
 25: 21.756547002367192,
 30: 20.020878664370425,
 35: 21.36290153855905}

In [12]:
nucl_2 = mda.Universe("PDB/nucl_DNA_H1_linkered.pdb")
base_coor = nucl_2.select_atoms(f"segid J and resid 10 15 20 25 30 35 and name C1'")
base_coor = base_coor.positions.tolist()
base_coor

[[-15.369000434875488, 75.06700134277344, 19.363000869750977],
 [1.8609999418258667, 85.62999725341797, 21.643999099731445],
 [-0.7580000162124634, 105.45700073242188, 17.94300079345703],
 [16.472000122070312, 116.02100372314453, 20.224000930786133],
 [13.852999687194824, 135.8470001220703, 16.52400016784668],
 [31.08300018310547, 146.41099548339844, 18.80500030517578]]

In [15]:
nucl_2 = mda.Universe("PDB/nucl_DNA_H1_linkered.pdb")
u_cube = mda.Universe("PDB/matrix_atoms.pdb")
base = nucl_2.select_atoms(f"segid J and resid 10 15 20 25 30 35")
# nv.show_mdanalysis(base)
merged = mda.Merge(
    base.atoms,
    # nucl_1.u.atoms
    u_cube.atoms,
)
merged.atoms.write(f"PDB/matrix_and_DNA_H1.pdb")

In [16]:
R0=53.0
E_101 = np.array([0.72, 0.76, 0.56, 0.56, 0.46, 0.39])
E_195 = np.array([0.43, 0.48, 0.66, 0.47, 0.52, 0.44])
Rda_101 = (((R0**6)/E_101)-R0**6)**(1/6)
Rda_195 = (((R0**6)/E_195)-R0**6)**(1/6)
Rda_101

array([45.28073406, 43.73631054, 50.91197888, 50.91197888, 54.43545504,
       57.10227452])

In [26]:
selection_list_101 = []
selection_list_195 = []
nucl_cube = mda.Universe("PDB/matrix_and_DNA_H1.pdb")
for i, n_r in enumerate([10,15,20,25,30,35]):
    acid_101 = nucl_cube.select_atoms(f" sphlayer {Rda_101[i]-data_analysis[n_r]} {Rda_101[i]+data_analysis[n_r]} resid {n_r} ")
    acid_195 = nucl_cube.select_atoms(f" sphlayer {Rda_195[i]-data_analysis[n_r]} {Rda_195[i]+data_analysis[n_r]} resid {n_r} ")
    selection_list_101.append(acid_101.select_atoms(' not nucleic'))
    selection_list_195.append(acid_195.select_atoms(' not nucleic'))

In [33]:
intersection_101 = selection_list_101[0] & selection_list_101[1] & selection_list_101[2] & selection_list_101[3]  & selection_list_101[4] 
intersection_195 = selection_list_195[0] & selection_list_195[1] & selection_list_195[2] & selection_list_195[3]  & selection_list_195[4]
intersection_101

<AtomGroup with 14646 atoms>

In [37]:
nucl_2 = mda.Universe("PDB/nucl_DNA_H1_linkered.pdb")
base = nucl_2.select_atoms(f"segid u")
# nv.show_mdanalysis(base)
merged_1 = mda.Merge(
    nucl_2.atoms,
    # nucl_1.u.atoms
    intersection_101.atoms,
)
# amin = merged_2.select_atoms('')
view = nv.show_mdanalysis(merged_1)
view

NGLWidget()

In [36]:
nucl_2 = mda.Universe("PDB/nucl_DNA_H1_linkered.pdb")
base = nucl_2.select_atoms(f"segid u")
# nv.show_mdanalysis(base)
merged_2 = mda.Merge(
    nucl_2.atoms,
    # nucl_1.u.atoms
    intersection_195.atoms,
)
# amin = merged_2.select_atoms('')
view = nv.show_mdanalysis(merged_2)
view

NGLWidget()

In [38]:
merged_1.atoms.write('PDB/theor_space_ac101.pdb')
merged_2.atoms.write('PDB/theor_space_ac195.pdb')